In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

RUNS = 65_001
# avg between 48.2 and 48.8
BURDEN_BABIES = 48.5
# mitchell inference (calibration)
BURDEN_PER_YEAR = 14
BURDEN_PER_DIVISION = 1.2
ABC_4D = True  # infer tau from abc (4d abc)

In [ ]:
# RANGES!!
"""
as of 23/01/2024 the ranges are: TO BE determined??
    s (i.e. old s/tau) : 0.01 to 0.2
    sigma (i.e. old sigma/tau) : 0.001 to 0.05
    mu: 0.1 to 30
    tau: 0.1 to 10
    N: 25'000 to 900'000 # for now we fix this to 10^5
"""
cells = np.ones(RUNS, dtype=int) * 100_000
rng = np.random.default_rng(seed=42)

if ABC_4D:
    taus = rng.uniform(0.1, 10, RUNS)
else:
    taus = np.ones(RUNS, dtype=int)
mu0s = rng.uniform(0.1, 30, RUNS)
s_means = rng.uniform(0.01, 0.2, RUNS)
s_stds = rng.uniform(0.001, 0.05, RUNS)

assert (s_means * taus).max() < 2
assert (s_stds * taus).max() < 0.5

In [ ]:
def compute_m_exp(cells: int) -> float:
    return BURDEN_BABIES / (2 * np.log((cells + 1)/2))

def compute_m_background(tau: float) -> float:
    # 2 because we assume no asymmetric divisions
    return BURDEN_PER_YEAR - 2 * BURDEN_PER_DIVISION / tau

def compute_std_per_division_from_std_per_year(std_per_year: float, tau: float) -> float:
    return std_per_year * tau

def compute_s_per_division_from_s_per_year(s_per_year: float, tau: float) -> float:
    return s_per_year * tau

In [ ]:
plt.hist([compute_m_background(tau) for tau in taus], label="mu background", bins=100)
plt.legend()
plt.show()

sns.pairplot(pd.DataFrame([mu0s, taus, s_means, s_stds, cells], index=["mu", "tau","s", "std", "N"]).T)
plt.show()

In [ ]:
with open("parameters.txt", "w") as f:
    for (tau, mu, mean, std, n) in np.column_stack((taus, mu0s, s_means, s_stds, cells)):
        c = int(n)
        f.write(
            f"-c {c} -y 82 -r 1 --tau {tau} --mu0 {mu} --mu-exp {compute_m_exp(c)} --mu-division 1.2 --mu-background {compute_m_background(tau)} --mean-std {compute_s_per_division_from_s_per_year(mean, tau)} {compute_std_per_division_from_std_per_year(std, tau)}\n"
        )

In [ ]:
%%bash
head parameters.txt

In [ ]:
%%bash --out runs
wc -l parameters.txt

In [ ]:
runs = int(runs.split(" ")[0])
assert RUNS == runs, f"{RUNS} vs {runs}"